In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2024-01-12T12:29:41.598186-05:00

Python implementation: CPython
Python version       : 3.7.11
IPython version      : 7.10.1

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 4.15.0-161-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 40
Architecture: 64bit



In [3]:
%watermark --gpu

GPU Info: 
  GPU 0: Tesla V100-DGXS-32GB
  GPU 1: Tesla V100-DGXS-32GB
  GPU 2: Tesla V100-DGXS-32GB
  GPU 3: Tesla V100-DGXS-32GB



In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import sklearn
import gc
import os
import xgboost as xgb

In [5]:
%watermark --iversions

numpy  : 1.21.6
xgboost: 1.6.2
pandas : 1.3.5
sklearn: 1.0.2



In [6]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [7]:
train = pd.read_csv('../input/train.csv.zip')
test = pd.read_csv('../input/test.csv.zip')
sample_submission = pd.read_csv('../input/sample_submission.csv.zip')

In [8]:
optuna_log = pd.read_csv('optuna_xgb_output_0.csv')
optuna_log.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.260580,0.518387,0.588129,logloss,0.803620,0.001024,19,73,binary:logistic,0.734509,gpu_hist,COMPLETE
1,1,0.272333,0.554771,0.386440,logloss,0.515043,0.004684,19,226,binary:logistic,0.740431,gpu_hist,COMPLETE
2,2,0.273310,0.001337,0.321878,logloss,0.224638,0.035482,25,235,binary:logistic,0.647692,gpu_hist,COMPLETE
3,3,0.213742,0.475032,0.907356,logloss,0.025882,0.019162,21,5,binary:logistic,0.939027,gpu_hist,COMPLETE
4,4,0.250605,7.983068,0.797204,logloss,1.452732,0.037789,8,12,binary:logistic,0.532211,gpu_hist,COMPLETE


In [9]:
optuna_log.value.max()

0.2834224878819652

In [10]:
max_value_row = optuna_log[optuna_log['value'] == optuna_log['value'].max()]

In [11]:
max_value_row

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
94,94,0.283422,0.654147,0.917583,logloss,4.645511,0.013001,7,194,binary:logistic,0.66007,gpu_hist,COMPLETE


In [12]:
columns = test.columns[1:]

In [13]:
X = train[columns].values
X_test = test[columns].values
Y = train.target.values

In [14]:
params = {'objective': 'binary:logistic',
 'tree_method': 'hist',
 'lambda': 4.645511,
 'alpha': 0.654147,
 'colsample_bytree': 0.917,
 'subsample': 0.66,
 'learning_rate': 0.013,
 'max_depth': 7,
 'min_child_weight': 194,
 'eval_metric': 'logloss'}

In [15]:
dtest = xgb.DMatrix(X_test, enable_categorical=True)

In [1]:
%%time
train_oof = np.zeros((X.shape[0], ))
test_preds = 0
train_oof.shape
num_round = 1000

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    dtrain = xgb.DMatrix(train_features, train_target, enable_categorical=True)
    dval = xgb.DMatrix(val_features, val_target, enable_categorical=True)
    
    model = xgb.train(params, dtrain, num_round)
    #model.set_param({'predictor': 'cpu_predictor'})
    val_pred = model.predict(dval)
    train_oof[val_index] = val_pred
    print("Fold normalized:", gini_normalized(val_target, val_pred))
    test_preds += model.predict(dtest)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

NameError: name 'np' is not defined

In [17]:
gini_normalized(Y, train_oof)

0.28346581036663465

In [18]:
%%time
params = {'objective': 'binary:logistic',
 'tree_method': 'gpu_hist',
 #'device':'cuda',
 'lambda': 4.645511,
 'alpha': 0.654147,
 'colsample_bytree': 0.917,
 'subsample': 0.66,
 'learning_rate': 0.013,
 'max_depth': 7,
 'min_child_weight': 194,
 'eval_metric': 'logloss'}

train_oof = np.zeros((X.shape[0], ))
test_preds = 0
train_oof.shape
num_round = 1000

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    dtrain = xgb.DMatrix(train_features, train_target, enable_categorical=True)
    dval = xgb.DMatrix(val_features, val_target, enable_categorical=True)
    
    model = xgb.train(params, dtrain, num_round)
    #model.set_param({'predictor': 'gpu_predictor'})
    val_pred = model.predict(dval)
    train_oof[val_index] = val_pred
    print("Fold normalized:", gini_normalized(val_target, val_pred))
    test_preds += model.predict(dtest)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1
Fold normalized: 0.2857341685206029
Fitting fold 2
Fold normalized: 0.2923887984192146
Fitting fold 3
Fold normalized: 0.27523881916887216
Fitting fold 4
Fold normalized: 0.2844621179904104
Fitting fold 5
Fold normalized: 0.2797413515106076
CPU times: user 5min 11s, sys: 3.86 s, total: 5min 15s
Wall time: 31.4 s


In [19]:
gini_normalized(Y, train_oof)

0.28329662478517637

In [20]:
roc_auc_score(Y, train_oof)

0.6416483115888524

In [21]:
sample_submission['target'] = test_preds

In [22]:
sample_submission.to_csv('../submissions/XGB_5_fold_optuna_best_3.csv.zip', index=False, compression='zip')

In [23]:
val_features = X[val_index]
val_target = Y[val_index]

In [24]:
dval = xgb.DMatrix(val_features, val_target, enable_categorical=True)

On the leaderbaord this submission scores 0.2791 on the public test set, and 0.28408on the private dataset.

In [25]:
%%time
shap_values = model.predict(dval, pred_contribs=True)

CPU times: user 1.28 s, sys: 332 ms, total: 1.61 s
Wall time: 1.61 s


In [26]:
shap_values.shape

(119042, 58)

In [27]:
np.abs(shap_values).sum(axis=0)

array([3.77797974e+03, 1.69239893e+03, 1.17771230e+04, 4.48138525e+03,
       9.72645215e+03, 4.97610303e+03, 5.65329395e+03, 2.17470898e+03,
       2.62467114e+03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.09453125e+04, 4.53101270e+03,
       7.89881592e+03, 1.74906830e+02, 6.46144678e+03, 5.03993408e+03,
       8.97555957e+03, 5.70123535e+03, 1.25791016e+03, 5.41536914e+03,
       6.24417603e+02, 5.54664551e+02, 1.61843103e+03, 4.84520117e+03,
       5.27585327e+02, 3.53954175e+03, 0.00000000e+00, 2.52670996e+03,
       2.57715039e+03, 1.21780554e+03, 1.81504941e+04, 2.76894019e+03,
       2.84395142e+03, 8.75968079e+02, 1.72568799e+03, 9.48456177e+02,
       6.46486389e+02, 5.17987305e+02, 8.47497559e+02, 7.89650879e+02,
       9.21966064e+02, 8.22010559e+02, 1.06422925e+03, 1.38781763e+03,
       7.58653259e+02, 7.01478516e+02, 1.28183582e+03, 2.65834167e+02,
       2.69060120e+02, 5.21981812e+02, 1.96371964e+02, 5.96262268e+02,
      

In [28]:
np.abs(shap_values).sum(axis=0)

array([3.77797974e+03, 1.69239893e+03, 1.17771230e+04, 4.48138525e+03,
       9.72645215e+03, 4.97610303e+03, 5.65329395e+03, 2.17470898e+03,
       2.62467114e+03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.09453125e+04, 4.53101270e+03,
       7.89881592e+03, 1.74906830e+02, 6.46144678e+03, 5.03993408e+03,
       8.97555957e+03, 5.70123535e+03, 1.25791016e+03, 5.41536914e+03,
       6.24417603e+02, 5.54664551e+02, 1.61843103e+03, 4.84520117e+03,
       5.27585327e+02, 3.53954175e+03, 0.00000000e+00, 2.52670996e+03,
       2.57715039e+03, 1.21780554e+03, 1.81504941e+04, 2.76894019e+03,
       2.84395142e+03, 8.75968079e+02, 1.72568799e+03, 9.48456177e+02,
       6.46486389e+02, 5.17987305e+02, 8.47497559e+02, 7.89650879e+02,
       9.21966064e+02, 8.22010559e+02, 1.06422925e+03, 1.38781763e+03,
       7.58653259e+02, 7.01478516e+02, 1.28183582e+03, 2.65834167e+02,
       2.69060120e+02, 5.21981812e+02, 1.96371964e+02, 5.96262268e+02,
      